In [1]:
%matplotlib inline

In [2]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import json

from matplotlib import  pyplot as plt
import ipywidgets as widgets

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr

from lavian_et_al_2025.imaging.imaging_classes import TwoPExperiment

In [3]:
master =  Path(r"/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data")
fish_list = list(master.glob("*_f*"))

In [4]:
for fish in fish_list:
    print(fish)
    suite2p_dir = fish / 'suite2p'
    plane_list = list(suite2p_dir.glob("*00*"))

    exp = TwoPExperiment(fish)
    fs = int(exp['imaging']['microscope_config']['scanning']['framerate'])
    n_rfs = 8

    n_blocks = exp['stimulus']['protocol']['receptive_fields']['v06_front_cols_flashes_1x8']['n_trials']

    for path in plane_list:

        try:
             suite2p_data = fl.load(path / "data_from_suite2p_cells.h5")
        except:
            suite2p_data = fl.load(path / "data_from_suite2p_unfiltered.h5")

        traces = fl.load(path / "filtered_traces.h5", "/detr")
        coords = suite2p_data['coords']
        anatomy = suite2p_data['anatomy_stack']
        
        try:
            sens_regs = fl.load(path / 'sensory_regressors.h5', '/regressors')
        except:
            sens_regs = fl.load(path / 'sensory_regressors_cells.h5', '/regressors')

        ######################### Part 1 - getting trial timing for each direction
        trial_times = np.zeros((n_rfs, n_blocks))
        for rf in range(n_rfs):
            tmp_reg = sens_regs[rf]
            trial_times[rf] = np.where(np.diff(tmp_reg) > 0)[0]

        len_rec, num_neurons = np.shape(traces)
        traces = traces.T

        ######################### Part 2 - chunking the traces into trials in a new way
        new_len_rec = len_rec // (n_rfs * n_blocks)
        trial_traces_new = np.zeros((num_neurons, n_blocks, new_len_rec*n_rfs))
        count = np.zeros((8), dtype=int)
        for i in range(n_blocks*n_rfs):
            t1 =  i * new_len_rec
            t2 = t1 + new_len_rec

            ### find rf type for current trial
            curr_rf = np.where(np.nanmean(sens_regs[:, t1:t2], axis=1) > 0.25)[0]

            t3 = int(curr_rf * new_len_rec)
            t4 = t3 + new_len_rec

            trial_traces_new[:, int(count[curr_rf]), t3:t4] = traces[:, t1:t2]
            count[curr_rf] += 1


        ######################### Part 3 - looking for neurons that reliably respond to the visual stimulus
        # selectnig reliable neruons 


        dt = 1 / fs
        traces_xr = xr.DataArray(
            data=trial_traces_new,                               #Adding the data
            dims=['roi', 'block', 't'],                #Defining name of the dimensions
            coords={                                   #Defining values at which each dimension wase valuated
                'roi':np.arange(trial_traces_new.shape[0]), 
                'block':np.arange(n_blocks),
                't':np.arange(trial_traces_new.shape[2])*dt
                }
            )
        reliability_arr_combined = reliability(np.swapaxes(traces_xr, 0, 2).values)

        rel_thresh = threshold_otsu(reliability_arr_combined)
        print("Reliability threshold: ", rel_thresh)

        d = {'reliability_arr_combined': reliability_arr_combined,
            'trial_traces': trial_traces_new}

        fl.save(path / 'reliability_index_arr_fixed.h5', d)
